# Goal:
Perform data carpentry on NBA advanced boxscore data and load data into the database.

Import libraries needed for data manipulation and database work

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import psycopg2
import getpass
from psycopg2.extensions import adapt, register_adapter, AsIs
pd.set_option('display.max_columns', 500)

Connect to the database

In [2]:
mypasswd = getpass.getpass()

# Then connects to the DB
conn = psycopg2.connect(database = 'cs20_group4', 
                              user = 'fhfrf', 
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)

# Then remove the password from computer memory
del mypasswd

········


In [3]:
cursor = conn.cursor()

Read in csv file into a dataframe and view results

In [3]:
nba_boxes = pd.read_csv('../Data/nba_advanced_boxscores.csv')

In [4]:
nba_boxes.head()

,boxscore,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,away_field_goals,away_free_throw_attempt_rate,away_free_throw_attempts,away_free_throw_percentage,away_free_throws,away_losses,away_minutes_played,away_offensive_rating,away_offensive_rebound_percentage,away_offensive_rebounds,away_personal_fouls,away_points,away_steal_percentage,away_steals,away_three_point_attempt_rate,away_three_point_field_goal_attempts,away_three_point_field_goal_percentage,away_three_point_field_goals,away_total_rebound_percentage,away_total_rebounds,away_true_shooting_percentage,away_turnover_percentage,away_turnovers,away_two_point_field_goal_attempts,away_two_point_field_goal_percentage,away_two_point_field_goals,away_wins,date,home_assist_percentage,home_assists,home_block_percentage,home_blocks,home_defensive_rating,home_defensive_rebound_percentage,home_defensive_rebounds,home_effective_field_goal_percentage,home_field_goal_attempts,home_field_goal_percentage,home_field_goals,home_free_throw_attempt_rate,home_free_throw_attempts,home_free_throw_percentage,home_free_throws,home_losses,home_minutes_played,home_offensive_rating,home_offensive_rebound_percentage,home_offensive_rebounds,home_personal_fouls,home_points,home_steal_percentage,home_steals,home_three_point_attempt_rate,home_three_point_field_goal_attempts,home_three_point_field_goal_percentage,home_three_point_field_goals,home_total_rebound_percentage,home_total_rebounds,home_true_shooting_percentage,home_turnover_percentage,home_turnovers,home_two_point_field_goal_attempts,home_two_point_field_goal_percentage,home_two_point_field_goals,home_wins,location,losing_abbr,losing_name,pace,winner,winning_abbr,winning_name
0,201810160BOS,52.9,18,8.3,5,100.7,77.4,41.0,0.420,87,0.391,34,0.264,23,0.609,14,0,240,83.4,12.2,6.0,20,87,7.7,8,0.299,26,0.192,5,46.1,47,0.448,14.1,16,61,0.475,29,0,"8:00 PM, October 16, 2018",50.0,21,8.2,5.0,83.4,87.8,43.0,0.490,97,0.433,42,0.144,14,0.714,10,0,240,100.7,22.6,12.0,20,105,6.7,7,0.381,37,0.297,11,53.9,55,0.509,11.9,14,60,0.517,31,1,"TD Garden, Boston, Massachusetts",PHI,Philadelphia 76ers,104.3,Home,BOS,Boston Celtics
1,201810160GSW,63.6,21,8.7,6,105.6,63.0,29.0,0.418,91,0.363,33,0.407,37,0.649,24,1,240,97.7,28.1,16.0,21,100,11.7,12,0.407,37,0.270,10,43.7,45,0.466,11.5,14,54,0.426,23,0,"10:30 PM, October 16, 2018",66.7,28,13.0,7.0,97.7,71.9,41.0,0.479,95,0.442,42,0.189,18,0.944,17,0,240,105.6,37.0,17.0,29,108,6.8,7,0.274,26,0.269,7,56.3,58,0.525,16.9,21,69,0.507,35,1,"Oracle Arena, Oakland, California",OKC,Oklahoma City Thunder,102.3,Home,GSW,Golden State Warriors
2,201810170CHO,61.9,26,7.4,4,109.2,83.6,46.0,0.576,85,0.494,42,0.235,20,0.750,15,0,240,110.2,25.6,11.0,25,113,4.9,5,0.400,34,0.412,14,58.2,57,0.602,18.3,21,51,0.549,28,1,"7:00 PM, October 17, 2018",51.2,21,17.6,9.0,110.2,74.4,32.0,0.533,92,0.446,41,0.239,22,0.636,14,1,240,109.2,16.4,9.0,19,112,7.8,8,0.413,38,0.421,16,41.8,41,0.551,9.8,11,54,0.463,25,0,"Spectrum Center, Charlotte, North Carolina",CHO,Charlotte Hornets,102.6,Away,MIL,Milwaukee Bucks
3,201810170DET,70.0,28,7.4,5,103.0,70.8,34.0,0.518,82,0.488,40,0.268,22,0.682,15,1,240,100.0,13.5,5.0,23,100,9.0,9,0.329,27,0.185,5,45.9,39,0.545,15.6,17,55,0.636,35,0,"7:00 PM, October 17, 2018",53.8,21,9.1,5.0,100.0,86.5,32.0,0.457,92,0.424,39,0.239,22,0.864,19,0,240,103.0,29.2,14.0,20,103,5.0,5,0.261,24,0.250,6,54.1,46,0.506,12.1,14,68,0.485,33,1,"Little Caesars Arena, Detroit, Michigan",BRK,Brooklyn Nets,100.0,Home,DET,Detroit Pistons
4,201810170HOU,69.2,36,6.8,3,108.8,83.3,40.0,0.582,98,0.531,52,0.224,22,0.773,17,0,240,127.2,32.6,14.0,25,131,7.8,8,0.255,25,0.400,10,59.3,54,0.608,10.0,12,73,0.575,42,1,"8:00 PM, October 17, 2018",53.8,21,9.6,7.0,127.2,67.4,29.0,0.511,92,0.424,39,0.261,24,0.750,18,1,240,108.8,16.7,8.0,22,112,7.8,8,0.522,48,0.333,16,40.7,37,0.546,9.

Convert date column into datetime format

In [5]:
nba_boxes.date = pd.to_datetime(nba_boxes.date)

Convert percentages from 0 to 100 to range instead from 0 to 1 (decimal format) for away and home game stats

In [6]:
nba_boxes.away_assist_percentage = nba_boxes.away_assist_percentage / 100
nba_boxes.away_block_percentage = nba_boxes.away_block_percentage / 100
nba_boxes.away_defensive_rebound_percentage = nba_boxes.away_defensive_rebound_percentage / 100
nba_boxes.away_offensive_rebound_percentage = nba_boxes.away_offensive_rebound_percentage / 100
nba_boxes.away_steal_percentage = nba_boxes.away_steal_percentage / 100
nba_boxes.away_total_rebound_percentage = nba_boxes.away_total_rebound_percentage / 100
nba_boxes.away_turnover_percentage = nba_boxes.away_turnover_percentage / 100

In [7]:
nba_boxes.home_assist_percentage = nba_boxes.home_assist_percentage / 100
nba_boxes.home_block_percentage = nba_boxes.home_block_percentage / 100
nba_boxes.home_defensive_rebound_percentage = nba_boxes.home_defensive_rebound_percentage / 100
nba_boxes.home_offensive_rebound_percentage = nba_boxes.home_offensive_rebound_percentage / 100
nba_boxes.home_steal_percentage = nba_boxes.home_steal_percentage / 100
nba_boxes.home_total_rebound_percentage = nba_boxes.home_total_rebound_percentage / 100
nba_boxes.home_turnover_percentage = nba_boxes.home_turnover_percentage / 100

View our results

In [8]:
nba_boxes.head()

,boxscore,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,away_field_goals,away_free_throw_attempt_rate,away_free_throw_attempts,away_free_throw_percentage,away_free_throws,away_losses,away_minutes_played,away_offensive_rating,away_offensive_rebound_percentage,away_offensive_rebounds,away_personal_fouls,away_points,away_steal_percentage,away_steals,away_three_point_attempt_rate,away_three_point_field_goal_attempts,away_three_point_field_goal_percentage,away_three_point_field_goals,away_total_rebound_percentage,away_total_rebounds,away_true_shooting_percentage,away_turnover_percentage,away_turnovers,away_two_point_field_goal_attempts,away_two_point_field_goal_percentage,away_two_point_field_goals,away_wins,date,home_assist_percentage,home_assists,home_block_percentage,home_blocks,home_defensive_rating,home_defensive_rebound_percentage,home_defensive_rebounds,home_effective_field_goal_percentage,home_field_goal_attempts,home_field_goal_percentage,home_field_goals,home_free_throw_attempt_rate,home_free_throw_attempts,home_free_throw_percentage,home_free_throws,home_losses,home_minutes_played,home_offensive_rating,home_offensive_rebound_percentage,home_offensive_rebounds,home_personal_fouls,home_points,home_steal_percentage,home_steals,home_three_point_attempt_rate,home_three_point_field_goal_attempts,home_three_point_field_goal_percentage,home_three_point_field_goals,home_total_rebound_percentage,home_total_rebounds,home_true_shooting_percentage,home_turnover_percentage,home_turnovers,home_two_point_field_goal_attempts,home_two_point_field_goal_percentage,home_two_point_field_goals,home_wins,location,losing_abbr,losing_name,pace,winner,winning_abbr,winning_name
0,201810160BOS,0.529,18,0.083,5,100.7,0.774,41.0,0.420,87,0.391,34,0.264,23,0.609,14,0,240,83.4,0.122,6.0,20,87,0.077,8,0.299,26,0.192,5,0.461,47,0.448,0.141,16,61,0.475,29,0,2018-10-16 20:00:00,0.500,21,0.082,5.0,83.4,0.878,43.0,0.490,97,0.433,42,0.144,14,0.714,10,0,240,100.7,0.226,12.0,20,105,0.067,7,0.381,37,0.297,11,0.539,55,0.509,0.119,14,60,0.517,31,1,"TD Garden, Boston, Massachusetts",PHI,Philadelphia 76ers,104.3,Home,BOS,Boston Celtics
1,201810160GSW,0.636,21,0.087,6,105.6,0.630,29.0,0.418,91,0.363,33,0.407,37,0.649,24,1,240,97.7,0.281,16.0,21,100,0.117,12,0.407,37,0.270,10,0.437,45,0.466,0.115,14,54,0.426,23,0,2018-10-16 22:30:00,0.667,28,0.130,7.0,97.7,0.719,41.0,0.479,95,0.442,42,0.189,18,0.944,17,0,240,105.6,0.370,17.0,29,108,0.068,7,0.274,26,0.269,7,0.563,58,0.525,0.169,21,69,0.507,35,1,"Oracle Arena, Oakland, California",OKC,Oklahoma City Thunder,102.3,Home,GSW,Golden State Warriors
2,201810170CHO,0.619,26,0.074,4,109.2,0.836,46.0,0.576,85,0.494,42,0.235,20,0.750,15,0,240,110.2,0.256,11.0,25,113,0.049,5,0.400,34,0.412,14,0.582,57,0.602,0.183,21,51,0.549,28,1,2018-10-17 19:00:00,0.512,21,0.176,9.0,110.2,0.744,32.0,0.533,92,0.446,41,0.239,22,0.636,14,1,240,109.2,0.164,9.0,19,112,0.078,8,0.413,38,0.421,16,0.418,41,0.551,0.098,11,54,0.463,25,0,"Spectrum Center, Charlotte, North Carolina",CHO,Charlotte Hornets,102.6,Away,MIL,Milwaukee Bucks
3,201810170DET,0.700,28,0.074,5,103.0,0.708,34.0,0.518,82,0.488,40,0.268,22,0.682,15,1,240,100.0,0.135,5.0,23,100,0.090,9,0.329,27,0.185,5,0.459,39,0.545,0.156,17,55,0.636,35,0,2018-10-17 19:00:00,0.538,21,0.091,5.0,100.0,0.865,32.0,0.457,92,0.424,39,0.239,22,0.864,19,0,240,103.0,0.292,14.0,20,103,0.050,5,0.261,24,0.250,6,0.541,46,0.506,0.121,14,68,0.485,33,1,"Little Caesars Arena, Detroit, Michigan",BRK,Brooklyn Nets,100.0,Home,DET,Detroit Pistons
4,201810170HOU,0.692,36,0.068,3,108.8,0.833,40.0,0.582,98,0.531,52,0.224,22,0.773,17,0,240,127.2,0.326,14.0,25,131,0.078,8,0.255,25,0.400,10,0.593,54,0.608,0.100,12,73,0.575,42,1,2018-10-17 20:00:00,0.538,21,0.096,7.0,127.2,0.674,29.0,0.511,92,0.424,39,0.261,24,0.750,18,1,240,108.8,0.167,8.0,22,1

In [9]:
nba_boxes.shape

(43605, 83)

In [10]:
nba_boxes.date.dtypes

dtype('<M8[ns]')

In [11]:
nba_boxes.columns

Index(['boxscore', 'away_assist_percentage', 'away_assists',
       'away_block_percentage', 'away_blocks', 'away_defensive_rating',
       'away_defensive_rebound_percentage', 'away_defensive_rebounds',
       'away_effective_field_goal_percentage', 'away_field_goal_attempts',
       'away_field_goal_percentage', 'away_field_goals',
       'away_free_throw_attempt_rate', 'away_free_throw_attempts',
       'away_free_throw_percentage', 'away_free_throws', 'away_losses',
       'away_minutes_played', 'away_offensive_rating',
       'away_offensive_rebound_percentage', 'away_offensive_rebounds',
       'away_personal_fouls', 'away_points', 'away_steal_percentage',
       'away_steals', 'away_three_point_attempt_rate',
       'away_three_point_field_goal_attempts',
       'away_three_point_field_goal_percentage',
       'away_three_point_field_goals', 'away_total_rebound_percentage',
       'away_total_rebounds', 'away_true_shooting_percentage',
       'away_turnover_percentage', 'away_tu

Define nba_advanced_boxscores table for our database

In [13]:
create_table = """
DROP TABLE IF EXISTS nba_advanced_boxscores;
CREATE TABLE nba_advanced_boxscores (
boxscore varchar(100) PRIMARY KEY
, away_assist_percentage numeric
, away_assists numeric
, away_block_percentage numeric
, away_blocks numeric
, away_defensive_rating numeric
, away_defensive_rebound_percentage numeric
, away_defensive_rebounds numeric
, away_effective_field_goal_percentage numeric
, away_field_goal_attempts numeric
, away_field_goal_percentage numeric
, away_field_goals numeric
, away_free_throw_attempt_rate numeric
, away_free_throw_attempts numeric
, away_free_throw_percentage numeric
, away_free_throws numeric
, away_losses numeric
, away_minutes_played numeric
, away_offensive_rating numeric
, away_offensive_rebound_percentage numeric
, away_offensive_rebounds numeric
, away_personal_fouls numeric
, away_points numeric
, away_steal_percentage numeric
, away_steals numeric
, away_three_point_attempt_rate numeric
, away_three_point_field_goal_attempts numeric
, away_three_point_field_goal_percentage numeric
, away_three_point_field_goals numeric
, away_total_rebound_percentage numeric
, away_total_rebounds numeric
, away_true_shooting_percentage numeric
, away_turnover_percentage numeric
, away_turnovers numeric
, away_two_point_field_goal_attempts numeric
, away_two_point_field_goal_percentage numeric
, away_two_point_field_goals numeric
, away_wins numeric 
, game_date TIMESTAMP
, home_assist_percentage numeric
, home_assists numeric
, home_block_percentage numeric
, home_blocks numeric
, home_defensive_rating numeric
, home_defensive_rebound_percentage numeric
, home_defensive_rebounds numeric
, home_effective_field_goal_percentage numeric
, home_field_goal_attempts numeric
, home_field_goal_percentage numeric
, home_field_goals numeric
, home_free_throw_attempt_rate numeric
, home_free_throw_attempts numeric
, home_free_throw_percentage numeric
, home_free_throws numeric
, home_losses numeric
, home_minutes_played numeric
, home_offensive_rating numeric
, home_offensive_rebound_percentage numeric
, home_offensive_rebounds numeric
, home_personal_fouls numeric
, home_points numeric
, home_steal_percentage numeric
, home_steals numeric
, home_three_point_attempt_rate numeric
, home_three_point_field_goal_attempts numeric
, home_three_point_field_goal_percentage numeric
, home_three_point_field_goals numeric
, home_total_rebound_percentage numeric
, home_total_rebounds numeric
, home_true_shooting_percentage numeric
, home_turnover_percentage numeric
, home_turnovers numeric
, home_two_point_field_goal_attempts numeric
, home_two_point_field_goal_percentage numeric
, home_two_point_field_goals numeric
, home_wins numeric
, location varchar(250)
, losing_abbr varchar(100)
, losing_name varchar(100)
, pace numeric
, winner varchar(100)
, winning_abbr varchar(100)
, winning_name varchar(100)
)
"""

Execute table definition SQL

In [14]:
cursor.execute(create_table)

In [15]:
conn.commit()

In [16]:
nba_boxes.shape

(43605, 83)

Insert not null data into our nba_advanced_boxscores table

In [17]:
nba_boxes = nba_boxes.where(pd.notnull(nba_boxes), None)

INSERT_SQL = 'INSERT INTO nba_advanced_boxscores'
INSERT_SQL += '(boxscore'
INSERT_SQL += ', away_assist_percentage'
INSERT_SQL += ', away_assists'
INSERT_SQL += ', away_block_percentage'
INSERT_SQL += ', away_blocks'
INSERT_SQL += ', away_defensive_rating'
INSERT_SQL += ', away_defensive_rebound_percentage'
INSERT_SQL += ', away_defensive_rebounds'
INSERT_SQL += ', away_effective_field_goal_percentage'
INSERT_SQL += ', away_field_goal_attempts'
INSERT_SQL += ', away_field_goal_percentage'
INSERT_SQL += ', away_field_goals'
INSERT_SQL += ', away_free_throw_attempt_rate'
INSERT_SQL += ', away_free_throw_attempts'
INSERT_SQL += ', away_free_throw_percentage'
INSERT_SQL += ', away_free_throws'
INSERT_SQL += ', away_losses'
INSERT_SQL += ', away_minutes_played'
INSERT_SQL += ', away_offensive_rating'
INSERT_SQL += ', away_offensive_rebound_percentage'
INSERT_SQL += ', away_offensive_rebounds'
INSERT_SQL += ', away_personal_fouls'
INSERT_SQL += ', away_points'
INSERT_SQL += ', away_steal_percentage'
INSERT_SQL += ', away_steals'
INSERT_SQL += ', away_three_point_attempt_rate'
INSERT_SQL += ', away_three_point_field_goal_attempts'
INSERT_SQL += ', away_three_point_field_goal_percentage'
INSERT_SQL += ', away_three_point_field_goals'
INSERT_SQL += ', away_total_rebound_percentage'
INSERT_SQL += ', away_total_rebounds'
INSERT_SQL += ', away_true_shooting_percentage'
INSERT_SQL += ', away_turnover_percentage'
INSERT_SQL += ', away_turnovers'
INSERT_SQL += ', away_two_point_field_goal_attempts'
INSERT_SQL += ', away_two_point_field_goal_percentage'
INSERT_SQL += ', away_two_point_field_goals'
INSERT_SQL += ', away_wins'
INSERT_SQL += ', game_date'
INSERT_SQL += ', home_assist_percentage'
INSERT_SQL += ', home_assists'
INSERT_SQL += ', home_block_percentage'
INSERT_SQL += ', home_blocks'
INSERT_SQL += ', home_defensive_rating'
INSERT_SQL += ', home_defensive_rebound_percentage'
INSERT_SQL += ', home_defensive_rebounds'
INSERT_SQL += ', home_effective_field_goal_percentage'
INSERT_SQL += ', home_field_goal_attempts'
INSERT_SQL += ', home_field_goal_percentage'
INSERT_SQL += ', home_field_goals'
INSERT_SQL += ', home_free_throw_attempt_rate'
INSERT_SQL += ', home_free_throw_attempts'
INSERT_SQL += ', home_free_throw_percentage'
INSERT_SQL += ', home_free_throws'
INSERT_SQL += ', home_losses'
INSERT_SQL += ', home_minutes_played'
INSERT_SQL += ', home_offensive_rating'
INSERT_SQL += ', home_offensive_rebound_percentage'
INSERT_SQL += ', home_offensive_rebounds'
INSERT_SQL += ', home_personal_fouls'
INSERT_SQL += ', home_points'
INSERT_SQL += ', home_steal_percentage'
INSERT_SQL += ', home_steals'
INSERT_SQL += ', home_three_point_attempt_rate'
INSERT_SQL += ', home_three_point_field_goal_attempts'
INSERT_SQL += ', home_three_point_field_goal_percentage'
INSERT_SQL += ', home_three_point_field_goals'
INSERT_SQL += ', home_total_rebound_percentage'
INSERT_SQL += ', home_total_rebounds'
INSERT_SQL += ', home_true_shooting_percentage'
INSERT_SQL += ', home_turnover_percentage'
INSERT_SQL += ', home_turnovers'
INSERT_SQL += ', home_two_point_field_goal_attempts'
INSERT_SQL += ', home_two_point_field_goal_percentage'
INSERT_SQL += ', home_two_point_field_goals'
INSERT_SQL += ', home_wins'
INSERT_SQL += ', location'
INSERT_SQL += ', losing_abbr'
INSERT_SQL += ', losing_name'
INSERT_SQL += ', pace'
INSERT_SQL += ', winner'
INSERT_SQL += ', winning_abbr'
INSERT_SQL += ', winning_name) VALUES'
INSERT_SQL += '(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s)'

with conn, conn.cursor() as cursor:
    for row in nba_boxes.itertuples(index=False, name=None):
        cursor.execute(INSERT_SQL,row)